In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-06-12 12:17:27.882389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 12:17:38.726394: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/macula/SMATousi/.conda/envs/mac-deep/lib/:/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/nvidia/cudnn/lib
2023-06-12 12:17:38.728248: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9469744745569415091
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10088022016
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14572668863915254932
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:19:00.0, compute capability: 7.5"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10088022016
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16271735185002732189
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5"
xla_global_id: 2144165316
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 10088022016
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1446188156585758040
physical_device_desc: "device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:67:00.0, compute capability: 7.5"
xla_global_id: 1651660799
, name: "/d

2023-06-12 12:18:05.432943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 12:18:11.508283: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:0 with 9620 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:19:00.0, compute capability: 7.5
2023-06-12 12:18:11.511080: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:1 with 9620 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-06-12 12:18:11.512196: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:2 with 9620 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [3]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()


2023-05-27 10:33:57.707973: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 14, 14, 32)   320         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 7, 7, 64)     18496       ['conv2d[0][0]']                 
                                                                                                  
 flatten (Flatten)              (None, 3136)         0           ['conv2d_1[0][0]']               
                                                                                            

In [4]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 3136)              9408      
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 64)       36928     
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 28, 28, 32)       18464     
 ranspose)                                                       
                                                                 
 conv2d_transpose_2 (Conv2DT  (None, 28, 28, 1)        289 

In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


In [6]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(mnist_digits, epochs=2, batch_size=128)


11490434/11490434 [==============================] - 2s 0us/step
Epoch 1/2
547/547 [==============================] - 66s 115ms/step - loss: 248.3755 - reconstruction_loss: 205.1570 - kl_loss: 3.4305
Epoch 2/2
547/547 [==============================] - 69s 126ms/step - loss: 178.6454 - reconstruction_loss: 167.3625 - kl_loss: 5.1407


In [4]:
import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "/home/macula/SMATousi/VAE_outlier/archive/creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)


HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


In [6]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))


Number of training samples: 227846
Number of validation samples: 56961


In [7]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


In [8]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

In [9]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


latent_dim = 2

encoder_inputs = keras.Input(shape=(30,))
# x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
# x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Flatten()(x)
x = layers.Dense(30, activation="tanh")(encoder_inputs)
x = layers.Dense(20, activation="tanh")(x)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(16, activation="tanh")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

latent_inputs = keras.Input(shape=(latent_dim,))
# x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
# x = layers.Reshape((7, 7, 64))(x)
# x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
# decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
x = layers.Dense(16, activation="tanh")(latent_inputs)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(20, activation="tanh")(x)
decoder_outputs = layers.Dense(30, activation="tanh")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")



2023-05-30 12:53:40.370089: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 12:53:40.370667: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
#             reconstruction_loss = tf.reduce_mean(
#                 tf.reduce_sum(
#                     keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
#                 )
#             )
            reconstruction_loss = tf.keras.losses.MeanSquaredError()(data,reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


In [11]:

creditdata = np.concatenate([train_features, val_features], axis=0)
creditdata = np.expand_dims(creditdata, -1).astype("float32")

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(creditdata, epochs=10, batch_size=128)

Epoch 1/10
2226/2226 [==============================] - 4s 1ms/step - loss: 1.0310 - reconstruction_loss: 1.0054 - kl_loss: 0.0035
Epoch 2/10
2226/2226 [==============================] - 2s 1ms/step - loss: 1.0149 - reconstruction_loss: 1.0047 - kl_loss: 2.8087e-05
Epoch 3/10
2226/2226 [==============================] - 3s 1ms/step - loss: 0.9983 - reconstruction_loss: 1.0048 - kl_loss: 7.8637e-06
Epoch 4/10
2226/2226 [==============================] - 3s 1ms/step - loss: 0.9902 - reconstruction_loss: 1.0045 - kl_loss: 3.5634e-06
Epoch 5/10
2226/2226 [==============================] - 3s 1ms/step - loss: 0.9902 - reconstruction_loss: 1.0044 - kl_loss: 2.1467e-06
Epoch 6/10
2226/2226 [==============================] - 3s 1ms/step - loss: 0.9975 - reconstruction_loss: 1.0044 - kl_loss: 1.5636e-06
Epoch 7/10
2226/2226 [==============================] - 3s 1ms/step - loss: 0.9956 - reconstruction_loss: 1.0043 - kl_loss: 1.2261e-06
Epoch 8/10
2226/2226 [==============================] - 3s 

NameError: name 'python' is not defined

In [67]:
test = train_features[1,:]

test = test.reshape([1,30])

In [77]:
z_mean, z_log_var, z = vae.encoder(test)
reconstruction = vae.decoder(z)

reconstruction_loss = tf.keras.losses.MeanSquaredError()(test,reconstruction)

In [78]:
reconstruction_loss.numpy()

0.41741496

In [70]:
reconstruction.numpy()

array([[ 0.41229054,  0.02760492,  0.00990844, -0.13348393, -0.03093056,
         0.04025487, -0.02018991,  0.01525421, -0.0075708 ,  0.0047708 ,
         0.01919036, -0.07557505,  0.04402744, -0.0281549 , -0.02722586,
        -0.04202577,  0.0039289 , -0.01101142,  0.02211242,  0.00563244,
        -0.01162461,  0.01399416,  0.0240715 ,  0.01843469, -0.0130402 ,
        -0.07930604, -0.01005057, -0.0105467 ,  0.00252189, -0.00742954]],
      dtype=float32)

In [71]:
test

array([[-2.0008564 ,  0.64784056,  0.16869006, -0.01075485,  0.28652602,
         0.09012804, -0.08543706, -0.04230526,  0.0670673 , -0.2293523 ,
        -0.15108545,  1.4866744 ,  1.0515386 ,  0.4633228 , -0.18291788,
         0.62796277,  0.5300628 , -0.14867269, -0.1904353 , -0.16889915,
        -0.10145202, -0.29439253, -0.8689419 ,  0.17637353, -0.56267697,
         0.25427756,  0.25284111, -0.0230066 ,  0.03819847, -0.3517778 ]],
      dtype=float32)

In [12]:
rec_loss = []

for i in range(val_features.shape[0]):
    
    if val_targets[i] == 1:
        
        test = val_features[i,:]

        test = test.reshape([1,30])

        z_mean, z_log_var, z = vae.encoder(test)
        reconstruction = vae.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(test,reconstruction)
        
        rec_loss.append(reconstruction_loss.numpy())


        print("Rec Loss = ", reconstruction_loss.numpy(), " -------- Label = ", val_targets[i], " Case No. ", i)

rec_loss = np.array(rec_loss)
print("mean = ", np.mean(rec_loss))
print("std = ", np.std(rec_loss))

Rec Loss =  7.5094476  -------- Label =  [1]  Case No.  1866
Rec Loss =  3.3196588  -------- Label =  [1]  Case No.  1884
Rec Loss =  10.030133  -------- Label =  [1]  Case No.  2230
Rec Loss =  6.4608707  -------- Label =  [1]  Case No.  2630
Rec Loss =  5.0940776  -------- Label =  [1]  Case No.  4132
Rec Loss =  5.3070607  -------- Label =  [1]  Case No.  5412
Rec Loss =  12.009609  -------- Label =  [1]  Case No.  6728
Rec Loss =  5.2277074  -------- Label =  [1]  Case No.  6786
Rec Loss =  5.2294545  -------- Label =  [1]  Case No.  6787
Rec Loss =  6.815138  -------- Label =  [1]  Case No.  6859
Rec Loss =  8.662293  -------- Label =  [1]  Case No.  7770
Rec Loss =  8.746953  -------- Label =  [1]  Case No.  7788
Rec Loss =  8.010825  -------- Label =  [1]  Case No.  7798
Rec Loss =  7.5274982  -------- Label =  [1]  Case No.  9261
Rec Loss =  4.9370375  -------- Label =  [1]  Case No.  9580
Rec Loss =  15.51634  -------- Label =  [1]  Case No.  10376
Rec Loss =  4.067612  ------

In [12]:
val_features[6728,:]

array([  1.7462541 ,  -0.50769436,   2.6397295 ,  -4.164056  ,
         3.7277195 ,   1.124849  ,  -1.683369  ,  -1.2245529 ,
         0.51499057,  -3.5956087 ,  -5.744471  ,   3.3477724 ,
        -7.4962306 ,  -1.657718  , -12.934678  ,  -0.2816221 ,
        -1.6018841 ,  -3.7281365 ,   0.09827992,  -2.1891088 ,
         0.669623  ,   0.54346186,  -1.3017883 ,  -0.5808547 ,
        -0.36568624,   0.4449785 ,   0.09471209,   2.115266  ,
         1.5862252 ,  -0.36251542], dtype=float32)

In [19]:
import pygad


def fitness_func(ga_instance, solution, solution_idx):
    
    particle = val_features[6728,:]
    particle = particle.reshape([1,30])
    
#     abn_subspace = solution * val_features[6728,:]
    
#     abn_subspace = abn_subspace.reshape([1,30])
    
    

    z_mean, z_log_var, z = vae.encoder(particle)
    reconstruction_1 = vae.decoder(z)
    
    replace = reconstruction_1 * solution
    
    abn_subspace = 1 - solution
    
    particle_abn = particle * abn_subspace
    
    particle_rec = particle_abn + replace
    
    
    z_mean, z_log_var, z = vae.encoder(particle_rec)
    reconstruction_1 = vae.decoder(z)
    
    rec_loss = tf.keras.losses.MeanSquaredError()(particle,particle_rec)
    fitness = rec_loss.numpy()
    
    return fitness


def on_generation(ga):
    print("Generation", ga.generations_completed)
    
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    
    print(solution_fitness)

In [53]:
def fitness_func_avg(ga_instance, solution, solution_idx):
    
    inliers = val_features[10:15,:]
    
    avg_ins = np.mean(val_features[10:15,:], axis=0)
    avg_ins = avg_ins.reshape([1,30])
    
    particle = val_features[6728,:]
    particle = particle.reshape([1,30])
    
#     abn_subspace = solution * val_features[6728,:]
    
#     abn_subspace = abn_subspace.reshape([1,30])

    avg_in_rec = []
    
    for index in range(inliers.shape[0]):
        
        candidate_inlier = inliers[index,:]
        candidate_inlier = candidate_inlier.reshape([1,30])
        
        in_remain = candidate_inlier * solution
        
        in_normal_subspace = 1 - solution
        
        in_replace = in_normal_subspace * avg_ins
        
        in_candidate = in_remain + in_replace
        
        z_mean, z_log_var, z = vae.encoder(in_candidate)
        in_candidate_rec = vae.decoder(z)
        
        
        rec_loss = tf.keras.losses.MeanSquaredError()(in_candidate,in_candidate_rec)
        
        avg_in_rec.append(rec_loss.numpy())
    
    avg_in_rec = np.array(avg_in_rec)
    avg_in_rec = np.mean(avg_in_rec)
        
        
    
    

#     z_mean, z_log_var, z = vae.encoder(particle)
#     reconstruction_1 = vae.decoder(z)
    
    out_remain = particle * solution
    
    out_normal_subspace = 1 - solution
    
    out_replace = avg_ins * out_normal_subspace
    
    out_candidate = out_remain + out_replace
    
    
    z_mean, z_log_var, z = vae.encoder(out_candidate)
    out_candidate_rec = vae.decoder(z)
    
    rec_loss = tf.keras.losses.MeanSquaredError()(out_candidate,out_candidate_rec)
    rec_loss = rec_loss.numpy()
    
    fitness = rec_loss / avg_in_rec
    
    return fitness

In [49]:
fitness_function = fitness_func_avg

num_generations = 50
num_parents_mating = 4

sol_per_pop = 100
num_genes = val_features[6728,:].shape[0]

init_range_low = -2
init_range_high = 5

parent_selection_type = "sss"
keep_parents = 1

space = [[0,1] for i in range(num_genes)]

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 10

In [50]:
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       on_generation=on_generation,
                       gene_space = space)

In [51]:
ga_instance.run()

Generation 1
37.11245
Generation 2
41.05748
Generation 3
45.728626
Generation 4
45.678024
Generation 5
45.5867
Generation 6
45.5867
Generation 7
45.998726
Generation 8
46.43835
Generation 9
46.453487
Generation 10
46.453487
Generation 11
46.453487
Generation 12
46.453487
Generation 13
46.453487
Generation 14
46.453487
Generation 15
47.00801
Generation 16
47.113224
Generation 17
47.113224
Generation 18
47.113224
Generation 19
47.503582
Generation 20
47.38894
Generation 21
47.38894
Generation 22
47.38894
Generation 23
47.38894
Generation 24
47.38894
Generation 25
47.38894
Generation 26
47.409275
Generation 27
47.38894
Generation 28
47.38894
Generation 29
47.38894
Generation 30
47.38894
Generation 31
47.38894
Generation 32
47.38894
Generation 33
47.38894
Generation 34
47.644253
Generation 35
47.424953
Generation 36
47.424953
Generation 37
47.424953
Generation 38
47.424953
Generation 39
47.424953
Generation 40
47.424953
Generation 41
47.424953
Generation 42
47.424953
Generation 43
47.42495

In [52]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

Parameters of the best solution : [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0.]
Fitness value of the best solution = 47.819488525390625


In [31]:
np.mean(val_features[10:15,:], axis=0)

array([ 1.67609   ,  0.29879466, -0.51317704,  0.03845291, -0.26599112,
       -0.4089473 , -0.11063142, -0.08701693,  0.02064687,  0.32344753,
       -0.24708238, -0.94346046,  0.38582838,  0.5944903 , -0.40336066,
       -0.1687354 ,  0.26231766, -0.567821  ,  0.17933348, -0.24484256,
        0.30691004,  0.12367544,  0.04679886, -0.19573799,  0.12336917,
       -0.37529045, -0.22803795, -0.13733166, -0.05003484,  0.4847347 ],
      dtype=float32)

In [40]:
val_features[10:15,:].shape

(5, 30)

In [9]:
val_dataset = np.random.rand(200, 30)

In [10]:
num_samples = 1000
num_dimensions = 30

# Generate random samples
random_samples = np.random.rand(num_samples, num_dimensions)

In [11]:
print(random_samples)

[[0.67743024 0.25345555 0.13721269 ... 0.52092047 0.40750846 0.20464901]
 [0.76032286 0.74206023 0.42532838 ... 0.71720405 0.98045262 0.39438384]
 [0.30459255 0.2779774  0.51034756 ... 0.64490894 0.89239919 0.15052649]
 ...
 [0.41204152 0.71944016 0.47328146 ... 0.40802448 0.39963379 0.85432651]
 [0.73030401 0.79193762 0.20377598 ... 0.58925518 0.05544492 0.89040338]
 [0.79458781 0.72340085 0.50311506 ... 0.22426583 0.99445266 0.70633326]]


In [12]:
out_dataset = random_samples


out_dataset[0,0] = 20
out_dataset[0,3] = 20
out_dataset[0,4] = 20
out_dataset[0,7] = 20
out_dataset[0,9] = 20
out_dataset[0,14] = 20
out_dataset[0,22] = 20


out_dataset[100,0] = 20
out_dataset[100,3] = 20
out_dataset[100,4] = 20
out_dataset[100,7] = 20
out_dataset[100,9] = 20
out_dataset[100,14] = 20
out_dataset[100,22] = 20

out_dataset[400,0] = 20
out_dataset[400,3] = 20
out_dataset[400,4] = 20
out_dataset[400,7] = 20
out_dataset[400,9] = 20
out_dataset[400,14] = 20
out_dataset[400,22] = 20

out_dataset[600,0] = 20
out_dataset[600,3] = 20
out_dataset[600,4] = 20
out_dataset[600,7] = 20
out_dataset[600,9] = 20
out_dataset[600,14] = 20
out_dataset[600,22] = 20

out_dataset[900,0] = 20
out_dataset[900,3] = 20
out_dataset[900,4] = 20
out_dataset[900,7] = 20
out_dataset[900,9] = 20
out_dataset[900,14] = 20
out_dataset[900,22] = 20

In [38]:
tf.random.set_seed(0)
np.random.seed(seed=0)

In [40]:


class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


latent_dim = 2

encoder_inputs = keras.Input(shape=(30,))
# x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
# x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Flatten()(x)
x = layers.Dense(30, activation="tanh")(encoder_inputs)
x = layers.Dense(20, activation="tanh")(x)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(16, activation="tanh")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

latent_inputs = keras.Input(shape=(latent_dim,))
# x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
# x = layers.Reshape((7, 7, 64))(x)
# x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
# decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
x = layers.Dense(16, activation="tanh")(latent_inputs)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(20, activation="tanh")(x)
decoder_outputs = layers.Dense(30, activation="tanh")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")



class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
#             reconstruction_loss = tf.reduce_mean(
#                 tf.reduce_sum(
#                     keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
#                 )
#             )
            reconstruction_loss = tf.keras.losses.MeanSquaredError()(data,reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

creditdata = np.concatenate([out_dataset, val_dataset], axis=0)
creditdata = np.expand_dims(creditdata, -1).astype("float32")

vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.legacy.Adam())
vae.fit(creditdata, epochs=100, batch_size=128)

Epoch 1/100
10/10 [==============================] - 1s 5ms/step - loss: 0.8828 - reconstruction_loss: 0.7192 - kl_loss: 0.1637
Epoch 2/100
10/10 [==============================] - 0s 6ms/step - loss: 0.7180 - reconstruction_loss: 0.8019 - kl_loss: 0.0652
Epoch 3/100
10/10 [==============================] - 0s 7ms/step - loss: 0.8998 - reconstruction_loss: 0.6515 - kl_loss: 0.0330
Epoch 4/100
10/10 [==============================] - 0s 7ms/step - loss: 0.6476 - reconstruction_loss: 0.6166 - kl_loss: 0.0202
Epoch 5/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5964 - reconstruction_loss: 0.5784 - kl_loss: 0.0136
Epoch 6/100
10/10 [==============================] - 0s 7ms/step - loss: 0.4159 - reconstruction_loss: 0.5364 - kl_loss: 0.0105
Epoch 7/100
10/10 [==============================] - 0s 7ms/step - loss: 0.4187 - reconstruction_loss: 0.7262 - kl_loss: 0.0086
Epoch 8/100
10/10 [==============================] - 0s 7ms/step - loss: 0.7496 - reconstruction_loss: 0

10/10 [==============================] - 0s 7ms/step - loss: 0.4303 - reconstruction_loss: 0.4287 - kl_loss: 2.6267e-04
Epoch 64/100
10/10 [==============================] - 0s 7ms/step - loss: 0.3074 - reconstruction_loss: 0.5440 - kl_loss: 2.6381e-04
Epoch 65/100
10/10 [==============================] - 0s 7ms/step - loss: 0.4503 - reconstruction_loss: 0.4286 - kl_loss: 2.6028e-04
Epoch 66/100
10/10 [==============================] - 0s 7ms/step - loss: 0.3802 - reconstruction_loss: 0.4290 - kl_loss: 2.4640e-04
Epoch 67/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5216 - reconstruction_loss: 0.4288 - kl_loss: 2.4529e-04
Epoch 68/100
10/10 [==============================] - 0s 7ms/step - loss: 0.4574 - reconstruction_loss: 0.4285 - kl_loss: 2.2977e-04
Epoch 69/100
10/10 [==============================] - 0s 7ms/step - loss: 0.3472 - reconstruction_loss: 0.4287 - kl_loss: 2.2316e-04
Epoch 70/100
10/10 [==============================] - 0s 7ms/step - loss: 0.2943 -

In [20]:
rec_loss = []

for i in range(val_dataset.shape[0]):
        
    test = val_dataset[i,:]

    test = test.reshape([1,30])

    z_mean, z_log_var, z = vae.encoder(test)
    reconstruction = vae.decoder(z)

    reconstruction_loss = tf.keras.losses.MeanSquaredError()(test,reconstruction)

    rec_loss.append(reconstruction_loss.numpy())


    print("Rec Loss = ", reconstruction_loss.numpy(), " Case No. ", i)

rec_loss = np.array(rec_loss)
print("mean = ", np.mean(rec_loss))
print("std = ", np.std(rec_loss))

Rec Loss =  0.07947672  Case No.  0
Rec Loss =  0.10691883  Case No.  1
Rec Loss =  0.08086084  Case No.  2
Rec Loss =  0.10728254  Case No.  3
Rec Loss =  0.08119939  Case No.  4
Rec Loss =  0.07774002  Case No.  5
Rec Loss =  0.06357469  Case No.  6
Rec Loss =  0.10001295  Case No.  7
Rec Loss =  0.08098818  Case No.  8
Rec Loss =  0.094823666  Case No.  9
Rec Loss =  0.07094186  Case No.  10
Rec Loss =  0.08174196  Case No.  11
Rec Loss =  0.08672131  Case No.  12
Rec Loss =  0.0656073  Case No.  13
Rec Loss =  0.07251198  Case No.  14
Rec Loss =  0.12106066  Case No.  15
Rec Loss =  0.10060452  Case No.  16
Rec Loss =  0.06803354  Case No.  17
Rec Loss =  0.06838033  Case No.  18
Rec Loss =  0.07226968  Case No.  19
Rec Loss =  0.08470681  Case No.  20
Rec Loss =  0.07705819  Case No.  21
Rec Loss =  0.07993095  Case No.  22
Rec Loss =  0.07472696  Case No.  23
Rec Loss =  0.08716389  Case No.  24
Rec Loss =  0.07195595  Case No.  25
Rec Loss =  0.05473205  Case No.  26
Rec Loss = 

In [16]:
mean_data = mean_data.reshape([1,30])

z_mean, z_log_var, z = vae.encoder(mean_data)
reconstruction = vae.decoder(z)

reconstruction_loss = tf.keras.losses.MeanSquaredError()(mean_data,reconstruction)

print(reconstruction_loss.numpy())

NameError: name 'mean_data' is not defined

In [17]:
test = out_dataset[3,:]

test = test.reshape([1,30])

z_mean, z_log_var, z = vae.encoder(test)
reconstruction = vae.decoder(z)

reconstruction_loss = tf.keras.losses.MeanSquaredError()(test,reconstruction)

# rec_loss.append(reconstruction_loss.numpy())


print("Rec Loss = ", reconstruction_loss.numpy(), " -------- Label = ", val_targets[i], " Case No. ", i)


NameError: name 'val_targets' is not defined

In [124]:
out_dataset = random_samples


out_dataset[0,0] = 0.2
out_dataset[0,3] = 0.2
out_dataset[0,4] = 0.2
out_dataset[0,7] = 0.2
out_dataset[0,9] = 0.2
out_dataset[0,14] = 0.2
out_dataset[0,22] = 0.2


out_dataset[100,0] = 20
out_dataset[100,3] = 20
out_dataset[100,4] = 20
out_dataset[100,7] = 20
out_dataset[100,9] = 20
out_dataset[100,14] = 20
out_dataset[100,22] = 20

out_dataset[400,0] = 20
out_dataset[400,3] = 20
out_dataset[400,4] = 20
out_dataset[400,7] = 20
out_dataset[400,9] = 20
out_dataset[400,14] = 20
out_dataset[400,22] = 20

out_dataset[600,0] = 20
out_dataset[600,3] = 20
out_dataset[600,4] = 20
out_dataset[600,7] = 20
out_dataset[600,9] = 20
out_dataset[600,14] = 20
out_dataset[600,22] = 20

out_dataset[900,0] = 20
out_dataset[900,3] = 20
out_dataset[900,4] = 20
out_dataset[900,7] = 20
out_dataset[900,9] = 20
out_dataset[900,14] = 20
out_dataset[900,22] = 20



inliers = out_dataset[5:80,:]
mean_data = np.mean(out_dataset, axis=0)
outlier1 = val_dataset[0,:]



# #------------------ replacing the genes here ---------------------

# inliers[:,1] = mean_data[1]

# outlier1[1] = mean_data[1]

# # ----------------------------------------------------------------

mean_mean = []

for step in range(10):

    outlier1 = outlier1.reshape([1,30])

    z_mean, z_log_var, z = vae.encoder(outlier1)
    reconstruction = vae.decoder(z)

    reconstruction_loss = tf.keras.losses.MeanSquaredError()(outlier1,reconstruction)

    print(reconstruction_loss.numpy())
    
    mean_mean.append(reconstruction_loss.numpy())

print("MEAN_MEAN = ", np.mean(np.array(mean_mean)))

0.077600114
0.08050948
0.07515724
0.081566714
0.08005253
0.08238045
0.06840949
0.073207274
0.08070919
0.07982394
MEAN_MEAN =  0.07794164


In [117]:
mean_mean = []

for step in range(10):

    mean_data = mean_data.reshape([1,30])

    z_mean, z_log_var, z = vae.encoder(mean_data)
    reconstruction = vae.decoder(z)

    reconstruction_loss = tf.keras.losses.MeanSquaredError()(mean_data,reconstruction)

    print(reconstruction_loss.numpy())
    
    mean_mean.append(reconstruction_loss.numpy())

print("MEAN_MEAN = ", np.mean(np.array(mean_mean)))

0.0011079184
0.00035862127
0.00019719994
0.00015978632
0.00019239521
0.0002868758
0.0004647649
0.00041914513
0.00020846368
0.0002250349
MEAN_MEAN =  0.00036202057


In [119]:
mean_mean = []

for step in range(10):

    mean_ins_error = []

    for index in range(inliers.shape[0]):

        cand = inliers[index,:]

        cand = cand.reshape([1,30])

        mean_data = mean_data.reshape([1,30])

        z_mean, z_log_var, z = vae.encoder(cand)
        reconstruction = vae.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(cand,reconstruction)

        mean_ins_error.append(reconstruction_loss.numpy())

    #     print("MEAN", index, " = ", reconstruction_loss.numpy())

    mean_ins_error = np.array(mean_ins_error)
    print("mean = ", np.mean(mean_ins_error))
    
    mean_mean.append(np.mean(mean_ins_error))
    

print("MEAN_MEAN = ", np.mean(np.array(mean_mean)))

mean =  0.08066361
mean =  0.08035765
mean =  0.080403335
mean =  0.08022572
mean =  0.080229364
mean =  0.080227144
mean =  0.08026449
mean =  0.0800722
mean =  0.07993385
mean =  0.08020754
MEAN_MEAN =  0.080258496


In [116]:
75.56239/0.08021

942.0569754394712

In [120]:
660+260+240+125+128+400

1813

In [87]:
import pygad 


out_dataset = random_samples


out_dataset[0,0] = 20
out_dataset[0,1] = 20
out_dataset[0,2] = 20
out_dataset[0,3] = 20
out_dataset[0,4] = 20
out_dataset[0,5] = 20
out_dataset[0,6] = 20


out_dataset[100,0] = 20
out_dataset[100,3] = 20
out_dataset[100,4] = 20
out_dataset[100,7] = 20
out_dataset[100,9] = 20
out_dataset[100,14] = 20
out_dataset[100,22] = 20

out_dataset[400,0] = 20
out_dataset[400,3] = 20
out_dataset[400,4] = 20
out_dataset[400,7] = 20
out_dataset[400,9] = 20
out_dataset[400,14] = 20
out_dataset[400,22] = 20

out_dataset[600,0] = 20
out_dataset[600,3] = 20
out_dataset[600,4] = 20
out_dataset[600,7] = 20
out_dataset[600,9] = 20
out_dataset[600,14] = 20
out_dataset[600,22] = 20

out_dataset[900,0] = 20
out_dataset[900,3] = 20
out_dataset[900,4] = 20
out_dataset[900,7] = 20
out_dataset[900,9] = 20
out_dataset[900,14] = 20
out_dataset[900,22] = 20


def fitness_func_avg(ga_instance, solution, solution_idx):
    
    inliers = out_dataset[10:15,:]
    
    avg_ins = np.mean(out_dataset[10:15,:], axis=0)
    avg_ins = avg_ins.reshape([1,30])
    
    particle = out_dataset[0,:]
    particle = particle.reshape([1,30])
    
#     abn_subspace = solution * val_features[6728,:]
    
#     abn_subspace = abn_subspace.reshape([1,30])

    avg_in_rec = []
    
    for index in range(inliers.shape[0]):
        
        candidate_inlier = inliers[index,:]
        candidate_inlier = candidate_inlier.reshape([1,30])
        
        in_remain = candidate_inlier * solution
        
        in_normal_subspace = 1 - solution
        
        in_replace = in_normal_subspace * avg_ins
        
        in_candidate = in_remain + in_replace
        
        z_mean, z_log_var, z = vae.encoder(in_candidate)
        in_candidate_rec = vae.decoder(z)
        
        
        rec_loss = tf.keras.losses.MeanSquaredError()(in_candidate,in_candidate_rec)
        
        avg_in_rec.append(rec_loss.numpy())
    
    avg_in_rec = np.array(avg_in_rec)
    avg_in_rec = np.mean(avg_in_rec)
        
        
    
    

#     z_mean, z_log_var, z = vae.encoder(particle)
#     reconstruction_1 = vae.decoder(z)
    
    out_remain = particle * solution
    
    out_normal_subspace = 1 - solution
    
    out_replace = avg_ins * out_normal_subspace
    
    out_candidate = out_remain + out_replace
    
    
    z_mean, z_log_var, z = vae.encoder(out_candidate)
    out_candidate_rec = vae.decoder(z)
    
    rec_loss = tf.keras.losses.MeanSquaredError()(out_candidate,out_candidate_rec)
    rec_loss = rec_loss.numpy()
    
    fitness = rec_loss / avg_in_rec
    
    return fitness

def on_generation(ga):
    print("Generation", ga.generations_completed)
    
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    
    print(solution_fitness)


fitness_function = fitness_func_avg

num_generations = 100
num_parents_mating = 4

sol_per_pop = 100
num_genes = out_dataset.shape[1]

init_range_low = -2
init_range_high = 5

parent_selection_type = "sss"
keep_parents = 1

space = [[0,1] for i in range(num_genes)]

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 20

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       on_generation=on_generation,
                       gene_space = space)

In [88]:
ga_instance.run()

Generation 1
3395.2341
Generation 2
3246.3584
Generation 3
3360.5186
Generation 4
3323.6565
Generation 5
3622.429
Generation 6
3622.429
Generation 7
3622.429
Generation 8
3622.429
Generation 9
3622.429
Generation 10
3622.429
Generation 11
3622.429
Generation 12
3622.429
Generation 13
3622.429
Generation 14
3622.429
Generation 15
3622.429
Generation 16
3622.429
Generation 17
3622.429
Generation 18
3622.429
Generation 19
3622.429
Generation 20
3622.429
Generation 21
3622.429
Generation 22
3622.429
Generation 23
3622.429
Generation 24
3622.429
Generation 25
3622.429
Generation 26
3622.429
Generation 27
3622.429
Generation 28
3622.429
Generation 29
3622.429
Generation 30
3622.429
Generation 31
3705.433
Generation 32
3702.5957
Generation 33
3702.5957
Generation 34
3702.5957
Generation 35
3977.9626
Generation 36
3964.604
Generation 37
3964.604
Generation 38
3964.604
Generation 39
3964.604
Generation 40
3964.604
Generation 41
3964.604
Generation 42
3964.604
Generation 43
3964.604
Generation 4

In [89]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

Parameters of the best solution : [1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 0.]
Fitness value of the best solution = 3964.60400390625


In [ ]:

out_dataset[900,0] = 20
out_dataset[900,3] = 20
out_dataset[900,4] = 20
out_dataset[900,7] = 20
out_dataset[900,9] = 20
out_dataset[900,14] = 20
out_dataset[900,22] = 20

# Creating the dataset

In [131]:
np.random.seed(0)

num_samples = 10000
num_dimensions = 30

# Generate random samples
raw_data = np.random.rand(num_samples, num_dimensions)

outlier_indices_1 = [0,1,2,3,4,5,6]
outlyin_amount_1 = 20
outlier_indices_2 = [0,2,4,6,8,10,12]
outlyin_amount_2 = 10
outlier_indices_3 = [0,3,6,9,12,15,18]
outlyin_amount_3 = 5
outlier_indices_4 = [0,4,8,12,16,20,24]
outlyin_amount_4 = 2
outlier_indices_5 = [0,5,10,15,20,25,29]
outlyin_amount_5 = 1.1

corrupted_data = raw_data

corrupted_data[100:120, outlier_indices_1] = outlyin_amount_1
corrupted_data[200:220, outlier_indices_2] = outlyin_amount_2
corrupted_data[300:320, outlier_indices_3] = outlyin_amount_3
corrupted_data[400:420, outlier_indices_4] = outlyin_amount_4
corrupted_data[500:520, outlier_indices_5] = outlyin_amount_5

# print(raw_data)

array([2.00000000e+01, 2.00000000e+01, 2.00000000e+01, 2.00000000e+01,
       2.00000000e+01, 2.00000000e+01, 2.00000000e+01, 5.34271818e-02,
       7.25594364e-01, 1.14274586e-02, 7.70580749e-01, 1.46946645e-01,
       7.95220826e-02, 8.96030342e-02, 6.72047807e-01, 2.45367210e-01,
       4.20539467e-01, 5.57368791e-01, 8.60551174e-01, 7.27044263e-01,
       2.70327905e-01, 1.31482799e-01, 5.53743204e-02, 3.01598634e-01,
       2.62118149e-01, 4.56140567e-01, 6.83281336e-01, 6.95625446e-01,
       2.83518847e-01, 3.79926956e-01])